# **morphOMICs_v2 Tutorial Notebook**
## Author: **Ryan Cubero**
### For questions, please contact <a href="mailto:rcubero@ist.ac.at">Ryan</a> <br/> For bugs and feature request, please go to the <a href="https://github.com/siegert-lab/MorphOMICs/issues">morphOMICs GitHub issues page</a> <br/>
#### In this notebook, we will go through the modular implementation of <code>morphOMICs</code>. The idea of modularization was inspired by many workflow frameworks where the user directs the library to perform a sequence of tasks with the indicated parameters.

<br> </br>
#### First, let's import all the necessary libraries to make the notebook work.

In [3]:
from morphomics import protocols
import tomli

2023-06-02 15:00:36.420336: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 15:00:37.382235: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-02 15:00:37.382281: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-02 15:00:40.559819: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

####  The central feature of <code>morphOMICs_v2</code> is the ***parameters file***. You can check out some examples of the parameter file in **<span style="color:#424242">morphOMICs_v2/Parameter_files</span>**. It is  written in <a href="https://toml.io/en/">TOML</a> which is then parsed into Python as a dictionary.

<br></br>
#### First, let's define the <code>parameters</code> dictionary by calling the <code>Protocols</code>.

#### <code>Protocols</code> defines the sequence of modules that will be executed and can be used to create workflows. You can choose the following executables: <code>Input</code>, <code>Load_data</code>, <code>Clean_frame</code>, <code>Bootstrap</code>, <code>Persistence_Images</code>, <code>UMAP</code>, <code>Palantir</code>, <code>Plotting</code>, <code>Mapping</code>, <code>Sholl_curves</code>, <code>Morphometrics</code>, <code>Prepare_ReductionInfo</code>, <code>Clear_morphframe</code>.

#### In the Protocol below, we shall perform a typical morphOMICs workflow, where first, we would like <code>morphOMICs_v2</code> to <code>Input</code> the 3D morphological reconstructions (as .swc files) and convert them into *TMD barcodes*. Then, we will <code>Clean_frame</code> to drop certain conditions that were loaded but are not needed for downstream analysis or rename some conditions. Then, we will <code>Bootstrap</code> and calculate <code>Persistence_Images</code>. Then, we perform dimensionality reduction using <code>UMAP</code>, prepare a .csv file that can be loaded with the _morphOMICs dashboard_ using <code>Prepare_ReductionInfo</code> and finally, output an interactive spectrum through <code>Plotting</code>.

In [4]:
parameters = tomli.loads(
    """
# sequential list of executables
# Choices of executables: Input, Load_data, Clean_frame, Bootstrap, Persistence_Images, UMAP, Palantir, iPyVolume_plotting, Mapping, Clear_morphframe, Sholl_curves, Morphometrics 
# This is an example of a standard morphOMICs pipeline to obtain the morphological spectrum
Protocols = [
        "Input",
        "Clean_frame",
        "Bootstrap",
        "Persistence_Images",
        "UMAP",
        "Prepare_ReductionInfo",
        "Plotting",
    ]
"""
)

<br> </br>
#### Now, we initialize the <code><span style="color:#ff8a80">_morphOMICs</span></code> class using <code>protocols.Protocols()</code>. Here, we set the <code><span style="color:#ff8a80">Parameters_ID</span></code> to any number. In my use-case, <code><span style="color:#ff8a80">Parameters_ID</span></code>  serves as an indicator for the set of parameters that I am using, which I write down on my notes. It helps make my results reproducible.

In [5]:
Parameters_ID = 1
_morphOMICs = protocols.Protocols(parameters, Parameters_ID)

# It will output the filename prefix for the files that will be subsequently saved

Unless you have specified the file prefix in the succeeding executables, 
this will be the file prefix: Morphomics.PID1


<br></br>
#### Then, we start with loading the data. <code><span style="color:#ff8a80">_morphOMICs.parameters</span></code> contains the parameters that we have initiated before and we should update it with <code>Input</code> parameters. I have placed comments on what information is needed.

In [6]:
# parameters to load 3D morphological reconstructions
_morphOMICs.parameters["Input"] = tomli.loads(
    """
# filepath pointing to the parent folder containing the .swc files arranged hierarchically according to conditions
"data_location_filepath" = "/media/drive_siegert/RyCu/Projects/TMD/_Data/Retina_sample"

# .swc file extension, "_corrected.swc" refers to .swc files that were corrected with NeurolandMLConverter
"extension" = "_corrected.swc"

# this must match the hierarchical structure of `data_location_filepath`
"conditions" = [
            "Region",
            "Condition",
            "Model",
            "Time",
            "Sex",
            "Animal",
        ]
        
# this is the TMD filtration function, can either be radial_distances, or path_distances
"barcode_filter" = "radial_distances"

# this is how the variable will be called
"morphoframe_name" = "morphoframe"

# I would advise saving the loaded data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true

# saving chunks of the whole data (according to `separated_by`) makes it less painful if there was a problem with the file processing in the middle of running `Input`
# this must be an element of `conditions`
"separated_by" = "Model"

# location where to save the data
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Data/_TMD_barcodes/Retina_sample"

# if 0, morphOMICs will automatically create a file prefix, i.e., Morphomics.PID[xxx].[barcode_filter].[separated_by]-
# Otherwise, this will be used as the file prefix
"file_prefix" = 0

"""
)

#### Let's run <code>Input</code>!

In [7]:
_morphOMICs.Input()

Loading the data from /media/drive_siegert/RyCu/Projects/TMD/_Data/Retina_sample
Saving dataset in /media/drive_siegert/RyCu/Projects/TMD/_Data/_TMD_barcodes/Retina_sample/Morphomics.PID1.TMD-radial_distances
You are now loading the 3D reconstructions (.swc files) from this folder: 
/media/drive_siegert/RyCu/Projects/TMD/_Data/Retina_sample

Sample filenames:
/media/drive_siegert/RyCu/Projects/TMD/_Data/Retina_sample/IPL/Adulthood/Ctrl_Kxa4h/Adult/F/CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL/Filament_005_IPL_cleared_Trace_0000_nl_corrected.swc
/media/drive_siegert/RyCu/Projects/TMD/_Data/Retina_sample/IPL/Adulthood/Ctrl_Kxa4h/Adult/F/CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL/Filament_005_IPL_cleared_Trace_0001_nl_corrected.swc
/media/drive_siegert/RyCu/Projects/TMD/_Data/Retina_sample/IPL/Adulthood/Ctrl_Kxa4h/Adult/F/CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL/Filament_005_IPL_cleared_Trace_0002_nl_corrected.swc
/media/drive_siegert/RyCu/Projects/TMD/_Data/Retina_sample/IPL/Ad

#### <code>Input</code> returns the instance <code><span style="color:#ff8a80">_morphOMICs.morphoframe</span></code> which is a dictionary of pandas DataFrame with keys indicated by <code><span style="color:#ff8a80">_morphOMICs.morphoframe[_morphOMICs.parameters["*Input*"]["*morphoframe_name*"]]</span></code> which contains the *TMD barcodes* (**Barcodes**).

In [8]:
_morphOMICs.morphoframe["morphoframe"].head()

,Region,Condition,Model,Time,Sex,Animal,_files,Filenames,Morphologies,Barcodes
0,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0000_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[20.75804684935549, 17.748095052710422], [14...."
1,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0001_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[78.46860149130009, 76.54534465922923], [57.4..."
2,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0002_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[34.33316491382563, 34.87596242399507], [26.1..."
3,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0003_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[122.92604691032585, 119.19617370956183], [66..."
4,IPL,Adulthood,Ctrl_Kxa4h,Adult,F,CRTLSAL4_F_iba488_cd68647_dapi_1_20161005_IPL,Filament_005_IPL_cleared_Trace_0004_nl_correct...,/media/drive_siegert/RyCu/Projects/TMD/_Data/R...,<morphomics.Neuron.Neuron.Neuron object at 0x7...,"[[66.91833092658726, 68.64142038885886], [64.8..."


#### We can also count the number of morphologies in each condition.

In [9]:
_morphOMICs.morphoframe["morphoframe"].pivot_table(
    index=["Region", "Model", "Sex"], aggfunc="size"
)

Region  Model        Sex
IPL     Ctrl_Kxa48h  F      34
                     M      36
        Ctrl_Kxa4h   F      23
                     M      29
OPL     Ctrl_Kxa48h  F      30
                     M      27
        Ctrl_Kxa4h   F      20
                     M      18
dtype: int64

<br></br>
#### Then, we clean out the _morphoframe_ to remove empty morphologies (which corresponds to artifacts during the Imaris semi-automated reconstructions) as well as renaming conditions using <code>Clean_frame</code>.

In [10]:
# criteria for cleaning the input files
_morphOMICs.parameters["Clean_frame"] = tomli.loads(
    """
# if not 0, must contain the filepath to the morphoframe which will then be saved into morphoframe_name
# otherwise, `morphoframe_name` is the morphoframe that will be cleaned up
"morphoframe_filepath" = 0
"morphoframe_name" = "morphoframe"

# remove morphologies if the number of bars is less than the cutoff
"barcode_size_cutoff" = 5

# retain bars whose length satisfy a certain cutoff
# must be an array with three elements, ["greater_than" "less_than" or "within", bar length condition (must be an array if using "within"), "drop" or "keep"]
# the example below keeps bars whose length is greater than 0, and less than 200.
# if not used, comment the elements of `barlength_cutoff` out
"barlength_cutoff" = [ 
#            ["less_than", 0, "drop"],
#            ["greater_than", 200, "drop"], 
#            ["within", [0,200], "keep"],
            ]

# enumerate which conditions will be merged
# must be an array with three elements [a header of the info_frame (is an element of `Input.conditions`),
#                                       a list of conditions that will be merged (must be an array), 
#                                       the new name of the merged conditions]
# if not used, comment the elements of `combine_conditions` out
"combine_conditions" = [
    ["Model", ["Ctrl_Kxa4h", "Ctrl_Kxa48h"], "Ctrl_Kxa"]
]

# enumerate restrictions
# must be an array with three elements [a header of the info_frame (is an element of `Input.conditions`),  
#                                       list of conditions to either drop or keep (must be an array), 
#                                       "drop" or "keep" conditions specified]
# if not used, comment the elements of `restrict_conditions` out
"restrict_conditions" = [
#    ["Region", ["GCL"], "drop"],
#    ["Model", ["rd1","Cx3cr1_het","Ctrl_Iso","Ctrl_Kxa","rd10",], "keep"],
]

# I would advise saving the cleaned data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true

# location where to save the data
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Data/_TMD_barcodes/Retina_sample"

# if 0, morphOMICs will automatically create a file prefix, i.e., Morphomics.PID[xxx].[barcode_filter].Cleaned.
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [11]:
_morphOMICs.Clean_frame()

In [12]:
_morphOMICs.morphoframe["morphoframe"].pivot_table(
    index=["Region", "Model", "Sex"], aggfunc="size"
)

Region  Model     Sex
IPL     Ctrl_Kxa  F      56
                  M      65
OPL     Ctrl_Kxa  F      49
                  M      45
dtype: int64

#### Notice that we have combined "Ctrl_Kxa4h" and "Ctrl_Kxa48h" into one condition.

<br></br>
#### Now, we <code>Bootstrap</code> the morphologies within  <code><span style="color:#ff8a80">_morphOMICs.morphoframe[_morphOMICs.parameters["*Input*"]["*morphoframe_name*"]]</span></code>.

In [13]:
# parameters for bootstrapping
_morphOMICs.parameters["Bootstrap"] = tomli.loads(
    """
# if not 0, must contain the filepath to the morphoframe
"morphoframe_filepath" = 0
"morphoframe_name" = "morphoframe"

# must be an array with two elements [feature (one of the headers in morphoframe), "bars", "scalar" or "array"]
"feature_to_bootstrap" = ["Barcodes", "bars"]           # bootstraps by collapsing the lists into a single list 
# "feature_to_bootstrap" = ["Barcode_length", "scalar"] # bootstraps by taking the average of an array
# "feature_to_bootstrap" = ["Sholl", "array"]           # # bootstraps by taking the element-wise average of a high-dimensional NumPy array

# use this if you want to constraint your bootstrapping to a subset of conditions that you did not drop using `Clean_frame`
# column name in morphoframe where the bootstrap_conditions are located
"condition_column" = "Model"
# conditions to bootstrap
# if this array is empty, it will bootstrap across all conditions in the morphoframe
"bootstrap_conditions" = [
#            "rd1",
#            "Cx3cr1_het",
#            "Ctrl_Iso",
            "Ctrl_Kxa",
#            "rd10",
        ]
        

# which conditions combinations which bootstrapping will consider as a unique condition
"bootstrap_resolution" = [
                    "Region",
                    "Model",
                    "Sex",
        ]
        
        
# set the seed of the random number, for reproducibility
"rand_seed" = 34151


# if this is opted, N_pop will be calculated as ratio*(total number of morphologies in a given condition combination)
# if 0, you must input N_pop
"ratio" = 0
# number of morphologies to take averages of
# set this to 1 if you don't want to perform bootstrapping
"N_pop" = 15
# number of bootstrap samples to create
"N_samples" = 500


# where the bootstrapped morphoframes will be stored
"bootstrapframe_name" = "bootstrap_frame"
"morphoinfo_name" = "bootstrap_info"


# I would advise saving the data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true
# path to folder where to store bootstrapped conditions
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Results/Retina_Morphomics/Retina_sample/"
# if 0, morphOMICs will automatically create a file prefix, i.e., Morphomics.PID[xxx].[barcode_filter].Bootstrap.
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [14]:
_morphOMICs.Bootstrap()

Performing bootstrapping for IPL_Ctrl_Kxa_F...
There are 56 morphologies to bootstrap...
Performing subsampling by random selection...
...done! 

Performing bootstrapping for IPL_Ctrl_Kxa_M...
There are 65 morphologies to bootstrap...
Performing subsampling by random selection...
...done! 

Performing bootstrapping for OPL_Ctrl_Kxa_F...
There are 49 morphologies to bootstrap...
Performing subsampling by random selection...
...done! 

Performing bootstrapping for OPL_Ctrl_Kxa_M...
There are 45 morphologies to bootstrap...
Performing subsampling by random selection...
...done! 



#### <code>Bootstrap</code> returns the instance <code><span style="color:#ff8a80">_morphOMICs.morphoframe[_morphOMICs.parameters["*Bootstrap*"]["*bootstrapframe_name*"]]</span></code> which contains the *TMD barcodes* (**Barcodes**).

In [15]:
_morphOMICs.morphoframe["bootstrap_frame"].head()

,Region,Model,Sex,Bootstrapped index,Barcodes
0,IPL,Ctrl_Kxa,F,"Int64Index([2, 9, 17, 18, 90, 99, 100, 102, 10...","[[34.33316491382563, 34.87596242399507], [26.1..."
1,IPL,Ctrl_Kxa,F,"Int64Index([0, 1, 6, 11, 12, 14, 15, 21, 90, 9...","[[20.75804684935549, 17.748095052710422], [14...."
2,IPL,Ctrl_Kxa,F,"Int64Index([2, 4, 6, 8, 9, 10, 11, 95, 96, 106...","[[34.33316491382563, 34.87596242399507], [26.1..."
3,IPL,Ctrl_Kxa,F,"Int64Index([1, 2, 8, 11, 12, 13, 16, 21, 96, 1...","[[78.46860149130009, 76.54534465922923], [57.4..."
4,IPL,Ctrl_Kxa,F,"Int64Index([0, 2, 12, 14, 17, 18, 19, 22, 91, ...","[[20.75804684935549, 17.748095052710422], [14...."


In [16]:
_morphOMICs.morphoframe["bootstrap_frame"].pivot_table(
    index=["Region", "Model", "Sex"], aggfunc="size"
)

Region  Model     Sex
IPL     Ctrl_Kxa  F      500
                  M      500
OPL     Ctrl_Kxa  F      500
                  M      500
dtype: int64

#### <code>Bootstrap</code> also returns a metadata (<code><span style="color:#ff8a80">_morphOMICs.metadata[_morphOMICs.parameters["*Bootstrap*"]["*morphoinfo_name*"]]</span></code>) which contains the conditions with which you performed <code>Bootstrap</code>.

In [17]:
_morphOMICs.metadata["bootstrap_info"]

,Region,Model,Sex
0,IPL,Ctrl_Kxa,F
1,IPL,Ctrl_Kxa,F
2,IPL,Ctrl_Kxa,F
3,IPL,Ctrl_Kxa,F
4,IPL,Ctrl_Kxa,F
...,...,...,...
1995,OPL,Ctrl_Kxa,M
1996,OPL,Ctrl_Kxa,M
1997,OPL,Ctrl_Kxa,M
1998,OPL,Ctrl_Kxa,M


<br></br>
#### Then, we calculate the <code>Persistence_Images</code> from the bootstrapped barcodes in <code><span style="color:#ff8a80">_morphOMICs.morphoframe[_morphOMICs.parameters["*Bootstrap*"]["*bootstrapframe_name*"]]</span></code>.

In [18]:
# parameters for calculating persistence images
_morphOMICs.parameters["Persistence_Images"] = tomli.loads(
    """
# if not 0, must contain the filepath to the bootstrapframe
"morphoframe_filepath" = 0
"morphoframe_name" = "bootstrap_frame"


# paramteres for calculating persistence images
# spread of the Gaussian
"bw_method" = 0.5
# how to normalize the persistence image, can be "sum" or "max"
"norm_method" = "sum"
# constraints to the axes limits
"xlims" = [0, 200]
"ylims" = [0, 200]
# if not 0, must point to the location containing the weights to each bar
"barcode_weight" = 0


# I would advise saving the data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true
# path to folder where to store bootstrapped conditions
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Results/Retina_Morphomics/Retina_sample/"
# if 0, morphOMICs will automatically create a file prefix, i.e., Morphomics.PID[xxx].[barcode_filter].Bootstrap.
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [19]:
_morphOMICs.Persistence_Images()

Calculating persistence images...
...done! 



#### Note that <code>Persistence_Images</code> creates a NumPy array containing the flattened persistence images (a persistence image is 100x100 pixels) and stores it in <code><span style="color:#ff8a80">_morphOMICs.metadata["PI_matrix"]</span></code>.

In [20]:
_morphOMICs.metadata["PI_matrix"].shape

(2000, 10000)

<br></br>
#### Now that we have calculated the persistence images, we can infer a lower dimensional manifold using <code>UMAP</code>.

In [21]:
# parameters for calculating persistence images
_morphOMICs.parameters["UMAP"] = tomli.loads(
    """
# if not 0, must contain the filepath to the bootstrapframe
"PersistenceImages_filepath" = 0


# if 0, no pixel filtering will be done
"filter_pixels" = 1
# if not 0, must contain the filepath to the indices of the persistence image that will be filtered
"filteredpixelindex_filepath" = 0
# if filteredpixedindex_filepath is not zero, this will be ignored
"pixel_std_cutoff" = 1e-4


# if 0, the persistence image array will not be initially reduced
# do not do this if you plan to use the UMAP function for mapping
"run_PCA" = 0
"n_PCs" = 50


# UMAP parameters
"n_neighbors" = 300
"min_dist" = 0.05
"spread" = 3.0
"random_state" = 34151
"metric" = "manhattan"
"densmap" = 0 # zero is false
"n_components" = 3


# I would advise saving the data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true
# path to folder where to store UMAP function and reduced manifold coordinates
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Results/Retina_Morphomics/Retina_sample/"
# if 0, morphOMICs will automatically create a file prefix
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [22]:
_morphOMICs.UMAP()

#### Running <code>UMAP</code> creates a NumPy array containing the coordinates of the inferred low-dimensional manifold and stores it in <code><span style="color:#ff8a80">_morphOMICs.metadata["X_umap"]</span></code>.

In [23]:
_morphOMICs.metadata["X_umap"].shape

(2000, 3)

<br></br>
#### Then, we prepare a _.csv file_ which summarizes the conditions and its coordinate in the UMAP space. You can upload this file in the <code>_morphOMICs dashboard_</code>.

In [24]:
# parameters for calculating persistence images
_morphOMICs.parameters["Prepare_ReductionInfo"] = tomli.loads(
    """
"declare_filepaths" = false
"UMAP_filepath" = 0
"BootstrapInfo_filepath" = 0

# dictionary keys to the metadata
"coordinate_key" = "X_umap"
"morphoinfo_key" = "bootstrap_info"
"coordinate_axisnames" = "UMAP"

# path to folder where to store UMAP function and reduced manifold coordinates
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Results/Retina_Morphomics/Retina_sample/"
# if 0, morphOMICs will automatically create a file prefix
# Otherwise, this will be used as the file prefix
"file_prefix" = 0
"""
)

In [25]:
_morphOMICs.Prepare_ReductionInfo()

<br></br>
#### Finally, we can visualize the UMAP manifold through an interactive 3D plot using <code>Plotting</code>, which takes advantage of the <code>ipyvolume</code> library. Note that you first need to fill out a **<span style="color:#607d8b">_colormap.csv_</span>** file in **<span style="color:#424242">morphOMICs_v2/Parameter_files</span>**

In [ ]:
# parameters for calculating persistence images
_morphOMICs.parameters["Plotting"] = tomli.loads(
    """
# if 0, you must supply Coordinate_filepath AND MorphoInfo_filepath
# else, you can either supply the ReductionInfo_filepath, 
# or if you want to infer the manifold coordinates and morphoinfo_frame from Protocol instance, supply any character other than '0'
"ReductionInfo_filepath" = [
"infer",
# reductioninfo_filepath2,
# reductioninfo_filepath3,
]
"coordinate_axisnames" = "UMAP"


"coordinate_key" = "X_umap"
"morphoinfo_key" = "bootstrap_info"


# If these are not 0, must point to the location of the manifold coordinates
"Coordinate_filepath" = [ 
# coordinate_filepath1,
# coordinate_filepath2,
# coordinate_filepath3,
]
# If these are not 0, must point to the location of the morpho_infoframe that corresponds to each element of `Coordinate_filepath`
"MorphoInfo_filepath" = [ 
# morphoinfo_filepath1,
# morphoinfo_filepath2,
# morphoinfo_filepath3,
]

# Location to the color mapping that will be used
# 
"colormap_filepath" = [
    "/media/drive_siegert/RyCu/Projects/TMD/_Tutorials/_morphOMICs_v2/colormap.1.csv",
# colapmap_filepath2,
# colapmap_filepath3,
]

# Must correspond to each element in `Coordinate_filepath`
label_prefixes = [
    "Sample"
# label_prefix2,
# label_prefix3,
]

# If you need to substitute the name of a condition in morpho_infoframe, use this
# 
"Substitutions" = [
#    [
#        ["Time", "Adult", "P30"], # replaces all instances of `Adult` in the `Time` column to `P30`
#        # replaces all instances of `Cx3cr1-het` and `Cx3cr1-hom` in `Model` column to `Development`
#        ["Model", "Cx3cr1_het",  "Development"], 
#        ["Model", "Cx3cr1_hom",  "Development"], 
#    ],
#   [ substitutions for morphoinfo_filepath2 ],
#   [ substitutions for morphoinfo_filepath3 ],
]

# Trigger to show the interactive plot
"show_plot" = true

# I would advise saving the data; value is either `true` or `false` (warning: take note that all the letters are in lower case)
"save_data" = true

# path to folder where to store UMAP function and reduced manifold coordinates
"save_folder" = "/media/drive_siegert/RyCu/Projects/TMD/_Results/Retina_Morphomics/Retina_sample/"

# if 0, morphOMICs will automatically create a file prefix
# Otherwise, this will be used as the file prefix
"file_prefix" = "Retina_3DSpectrum"
"""
)

In [29]:
_morphOMICs.Plotting()

The following file was not found: infer
Inferring the morpho_infoframe and manifold coordinates from Protocol class.
No substitutions for coordinate set 1...


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

<br></br>
## Congratulations! You have now completed the typical morphOMICs pipeline!

### A copy of the parameters file is saved in **<span style="color:#424242">morphOMICs_v2/Tutorial/Morpphomics.Parameters.1.toml</span>**